<a href="https://colab.research.google.com/github/vivekk2k28/Machine_Learning/blob/main/Natural%20Language%20Processing/Named%20Entity%20Recognition/Custom%20NER%20with%20spacy/Custom_ner_model_on_Codes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re

In [2]:
template=[
    "The patient's insurance plan covers the cost of HCPCS code {HCPCS_CODE}, a nebulizer machine, prescribed for managing chronic respiratory conditions.",
    "Our clinic offers assistance in obtaining DME code {DME_CODE}, a motorized scooter, to enhance mobility for patients with limited ambulation.",
    "Following the outpatient procedure for cataract removal CPT code {CPT_CODE}, the patient was provided with HCPCS code {HCPCS_CODE}, a protective eye shield, for post-operative care.",
    "The patient was diagnosed with hypertension ICD-10 code {ICD_CODE} and prescribed medication (HCPCS code: {HCPCS_CODE}) to manage blood pressure levels effectively.",
    "The patient's recent diagnosis of osteoarthritis ICD-10 code {ICD_CODE} necessitated the use of DME code {DME_CODE}, a knee brace, to provide stability and support.",
    "The clinic offers counseling services CPT code {CPT_CODE} for mental health support, tailored to the individual needs of each patient.",
    "The patient's treatment plan includes surgery for gallbladder removal CPT code {CPT_CODE}, scheduled for  to address recurrent episodes of cholecystitis.",
    "The patient received a prescription for HCPCS code {HCPCS_CODE}, a continuous glucose monitoring system, to manage diabetes mellitus (ICD-10 code: E11.9) effectively.",
    "The clinic provides education sessions CPT code{CPT_CODE} on proper nutrition and lifestyle modifications to support patients with diabetes (ICD-10 code: E11.9).",
    "The patient's recent diagnosis of obstructive sleep apnea (ICD-10 code {ICD_CODE}) led to the recommendation of DME code {DME_CODE}, a CPAP machine, for nocturnal respiratory support.",
    "The patient's insurance plan covers the cost of DME code {DME_CODE}, a stationary oxygen concentrator, for home use.",
    "The patient underwent HCPCS code {HCPCS_CODE}, an established patient office visit, during which the physician performed a comprehensive examination and reviewed the patient's medical history.",
    "Our clinic provides assistance in obtaining DME code {DME_CODE}, a standard electric wheelchair, for eligible patients.",
    "The patient underwent surgery for knee replacement ICD-10 code {ICD_CODE} and was prescribed a knee immobilizer for post-operative support.",
    "The patient's insurance plan covers the cost of DME code {DME_CODE} for SreeLekha, prescribed to aid in their rehabilitation following a motor vehicle accident.",
    "Following the surgical procedure for (CPT code: {CPT_CODE}), the patient was provided with HCPCS code {HCPCS_CODE}, a post-operative compression garment, for wound healing support.",
    "Our clinic offers counseling services (CPT code: {CPT_CODE}) for  aimed at addressing stress and anxiety related to their recent diagnosis .",
    "The patient's treatment plan includes the use of DME code {DME_CODE}, a walker, to assist with mobility during their recovery from a procedure.",
    "The prescription for HCPCS code {HCPCS_CODE} was filled for jackson, who requires ongoing management of the condition as per their recent ICD-10 diagnosis ({ICD_CODE}).",
    "Following the outpatient procedure for procedure (CPT code: {CPT_CODE}), the patient was instructed to use DME code {DME_CODE}, a TENS unit, for pain management.",
    "The patient's recent diagnosis of Headache (ICD-10 code: {ICD_CODE}) necessitates the use of HCPCS code {HCPCS_CODE}, a medical brace, to provide support and stability.",
    "Our clinic provides education sessions (CPT code: {CPT_CODE}) on lifestyle modifications for Deva, focusing on dietary changes and physical activity to manage their the condition.",
    "The patient underwent surgery for respiratory surgeer (CPT code: {CPT_CODE}), and HCPCS code {HCPCS_CODE} was provided post-operatively to assist with pain management.",
    "The recent exacerbation of the condition (ICD-10 code: {ICD_CODE}) requires the use of DME code {DME_CODE}, a portable oxygen concentrator, for improved respiratory function.",
]


In [6]:
import random

def generate_dme_code():
    # Define the range of DME codes
    dme_code_start = 100
    dme_code_end = 8002

    # Generate a random DME code within the specified range
    dme_code = "E" + str(random.randint(dme_code_start, dme_code_end)).zfill(4)

    return dme_code



# Example usage
print("Generated DME Code:", generate_dme_code())



Generated DME Code: E6511


In [8]:

def generate_cpt_code():
    # Define the range of CPT codes
    cpt_code_start = 100
    cpt_code_end = 99499

    # Generate a random CPT code within the specified range
    cpt_code = str(random.randint(cpt_code_start, cpt_code_end)).zfill(5)

    return cpt_code

# Example usage
print("Generated CPT Code:", generate_cpt_code())

Generated CPT Code: 31766


In [10]:
def generate_icd_code():
    # Define the range of letters for the first character (A-Z)
    first_char = random.choice([chr(i) for i in range(ord('A'), ord('Z')+1)])

    # Generate two random digits for the second and third characters
    second_char = random.randint(0, 9)

    third_char = random.randint(0, 9)

    # Combine the characters to form the ICD code
    icd_code = f"{first_char}{second_char}{third_char}"

    return icd_code

# Example usage
print("Generated ICD Code:", generate_icd_code())

Generated ICD Code: X72


In [17]:
def generate_hcpcs_code():
    # Generate a random uppercase alphabetical letter for the first character
    first_char = random.choice('ABCDEFGHIJKLMNOPQRSTUVWXYZ')

    # Generate four random digits for the numeric portion of the code
    numeric_part = ''.join(random.choices('0123456789', k=4))

    # Combine the characters to form the HCPCS code
    hcpcs_code = f"{first_char}{numeric_part}"

    return hcpcs_code

# Example usage
print("Generated HCPCS Code:", generate_hcpcs_code())

Generated HCPCS Code: E4038


In [18]:
place_holder_generator={
    "DME_CODE":generate_dme_code,
    "CPT_CODE":generate_cpt_code,
    "ICD_CODE":generate_icd_code,
    "HCPCS_CODE":generate_hcpcs_code
}
print("placeholder",place_holder_generator)

placeholder {'DME_CODE': <function generate_dme_code at 0x7852eff99750>, 'CPT_CODE': <function generate_cpt_code at 0x7852eff99990>, 'ICD_CODE': <function generate_icd_code at 0x7852eff99bd0>, 'HCPCS_CODE': <function generate_hcpcs_code at 0x7852eff99870>}


In [27]:
import re
def replace_placeholders(template, placeholder_to_generator):
    updated_text = template
    entities = []

    # Iterate over each placeholder in the template text
    while "{" in updated_text:
        # Find the next placeholder in the updated text
        match = re.search(r"\{([^{}]*)\}", updated_text)
        print("match",match)
        if not match:
            break
        placeholder = match.group(1)
        print("placeholder",placeholder)

        # Replace the placeholder with the generated value
        fake_value = placeholder_to_generator[placeholder]()
        print("fake_value",fake_value)
        start, end = match.span()
        print("start",start)
        print("end",end)
        updated_text = updated_text[:start] + fake_value + updated_text[end:]
        print("updated_text",updated_text)

        # # Update the start and end positions for entities
        # print("Placeholder:", placeholder)
        # print("Start:", start)
        # print("End:", start + len(fake_value))
        entities.append((start, start + len(fake_value), placeholder))
        #entities.append((start, end, placeholder))

    return updated_text, {"entities": entities}






In [28]:
def generate_training_data(placeholder_to_generator, templates, num_samples):
    training_data = []

    for _ in range(num_samples):
        template = random.choice(templates)
        updated_text, entities = replace_placeholders(
            template, placeholder_to_generator
        )
        training_data.append((updated_text, entities))

    return training_data


generate_training_data(place_holder_generator,template,5000)



Streaming output truncated to the last 5000 lines.
end 109
updated_text Following the surgical procedure for (CPT code: 10680), the patient was provided with HCPCS code Z7687, a post-operative compression garment, for wound healing support.
match <re.Match object; span=(65, 75), match='{CPT_CODE}'>
placeholder CPT_CODE
fake_value 78003
start 65
end 75
updated_text Following the outpatient procedure for cataract removal CPT code 78003, the patient was provided with HCPCS code {HCPCS_CODE}, a protective eye shield, for post-operative care.
match <re.Match object; span=(113, 125), match='{HCPCS_CODE}'>
placeholder HCPCS_CODE
fake_value P2586
start 113
end 125
updated_text Following the outpatient procedure for cataract removal CPT code 78003, the patient was provided with HCPCS code P2586, a protective eye shield, for post-operative care.
match <re.Match object; span=(51, 63), match='{HCPCS_CODE}'>
placeholder HCPCS_CODE
fake_value B6741
start 51
end 63
updated_text The patient received a

[("The patient's recent diagnosis of Headache (ICD-10 code: O57) necessitates the use of HCPCS code I5192, a medical brace, to provide support and stability.",
  {'entities': [(57, 60, 'ICD_CODE'), (97, 102, 'HCPCS_CODE')]}),
 ('Following the outpatient procedure for cataract removal CPT code 00157, the patient was provided with HCPCS code Q8601, a protective eye shield, for post-operative care.',
  {'entities': [(65, 70, 'CPT_CODE'), (113, 118, 'HCPCS_CODE')]}),
 ('The patient underwent surgery for knee replacement ICD-10 code Q09 and was prescribed a knee immobilizer for post-operative support.',
  {'entities': [(63, 66, 'ICD_CODE')]}),
 ('Our clinic provides assistance in obtaining DME code E3539, a standard electric wheelchair, for eligible patients.',
  {'entities': [(53, 58, 'DME_CODE')]}),
 ("The patient's insurance plan covers the cost of DME code E0148 for SreeLekha, prescribed to aid in their rehabilitation following a motor vehicle accident.",
  {'entities': [(57, 62, 'DME_C

In [29]:
import os
import pickle

folder_path="training_data"
if not os.path.exists(folder_path):
  os.makedirs(folder_path)

pickle_file_path=os.path.join(folder_path,"spacy_training_data.pkl")

training_data=generate_training_data(place_holder_generator,template,5000)

with open(pickle_file_path,"wb") as file:
  pickle.dump(training_data,file)

Streaming output truncated to the last 5000 lines.
end 65
updated_text The recent exacerbation of the condition (ICD-10 code: M32) requires the use of DME code {DME_CODE}, a portable oxygen concentrator, for improved respiratory function.
match <re.Match object; span=(89, 99), match='{DME_CODE}'>
placeholder DME_CODE
fake_value E2170
start 89
end 99
updated_text The recent exacerbation of the condition (ICD-10 code: M32) requires the use of DME code E2170, a portable oxygen concentrator, for improved respiratory function.
match <re.Match object; span=(57, 67), match='{DME_CODE}'>
placeholder DME_CODE
fake_value E5644
start 57
end 67
updated_text The patient's insurance plan covers the cost of DME code E5644, a stationary oxygen concentrator, for home use.
match <re.Match object; span=(56, 66), match='{ICD_CODE}'>
placeholder ICD_CODE
fake_value V21
start 56
end 66
updated_text The patient was diagnosed with hypertension ICD-10 code V21 and prescribed medication (HCPCS code: {HCPCS_CODE

In [23]:
! pip install -U spacy



In [24]:
! python -m spacy download en_core_web_lg


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 655.8 kB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
from spacy.tokens import DocBin
import random

nlp=spacy.load("en_core_web_lg")
with open ('./training_data/spacy_training_data.pkl','rb') as file:
  data=pickle.load(file)

# Shuffle the data
random.shuffle(data)

#split the data into 80% training and 20% testing
split_ratio=int(len(data)*0.8)

train_data=data[:split_ratio]
test_data=data[split_ratio:]

print("length of training_data",len(train_data))
print("length of test_data",len(test_data))

db = DocBin()
for sample in train_data:
    print("sample(0)",sample[0])
    print("sample(1)",sample[1])
    doc = nlp(sample[0])
    entities = []
    for start, end, label in sample[1]["entities"]:
        span = doc.char_span(start, end, label=label)
        if span is not None:
            entities.append(span)
    doc.ents = entities
    db.add(doc)
db.to_disk("./training_data/train.spacy")
print("Training data prepared")
# the DocBin will store the example documents --test
db_test = DocBin()
for sample in test_data:
    doc = nlp(sample[0])
    entities = []
    for start, end, label in sample[1]["entities"]:
        span = doc.char_span(start, end, label=label)
        if span is not None:
            entities.append(span)
    doc.ents = entities
    db_test.add(doc)
db_test.to_disk("./training_data/test.spacy")
print("Testing data prepared")

print("Preprocessing done")

Streaming output truncated to the last 5000 lines.
sample(1) {'entities': [(56, 59, 'ICD_CODE'), (99, 104, 'HCPCS_CODE')]}
sample(0) Our clinic offers counseling services (CPT code: 92924) for  aimed at addressing stress and anxiety related to their recent diagnosis .
sample(1) {'entities': [(49, 54, 'CPT_CODE')]}
sample(0) Following the outpatient procedure for procedure (CPT code: 12457), the patient was instructed to use DME code E4661, a TENS unit, for pain management.
sample(1) {'entities': [(60, 65, 'CPT_CODE'), (111, 116, 'DME_CODE')]}
sample(0) Our clinic provides assistance in obtaining DME code E2662, a standard electric wheelchair, for eligible patients.
sample(1) {'entities': [(53, 58, 'DME_CODE')]}
sample(0) The patient received a prescription for HCPCS code V3697, a continuous glucose monitoring system, to manage diabetes mellitus (ICD-10 code: E11.9) effectively.
sample(1) {'entities': [(51, 56, 'HCPCS_CODE')]}
sample(0) Our clinic provides education sessions (CPT code: 

In [ ]:
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy train config.cfg --output ./output --paths.train ./training_data/train.spacy --paths.dev ./training_data/test.spacy

✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     41.06    0.00    0.00    0.00    0.00
  0     200          0.76    508.42  100.00  100.00  100.00    1.00
  0     400          0.00      0.03  100.00  100.00  100.00    1.00
  0     600          2.85     12.93  100.00  100.00  100.00    1.00
  1     800         15.65     30.36  100.00  100.00  100.00    1.00
  1    1000          0.00      0.00  100.00  100.00  100.00    1.00
  2    1200          0.00      0.00  100.00  100.00  100.00    1.00
  2    1400         14.19     12.12   99.93   99.93   99.93    1

In [ ]:
!python -m spacy evaluate ./output/model-best/ ./training_data/test.spacy

ℹ Using CPU

================================== Results ==================================

TOK     100.00
NER P   100.00
NER R   100.00
NER F   100.00
SPEED   2518  


=============================== NER (per type) ===============================

                  P        R        F
ICD_CODE     100.00   100.00   100.00
DME_CODE     100.00   100.00   100.00
HCPCS_CODE   100.00   100.00   100.00
CPT_CODE     100.00   100.00   100.00



In [ ]:
# import os
# output_dir="/content/model-last"
# if not os.path.exists(output_dir):
#   os.makedirs(output_dir)

# nlp.to_disk(output_dir)

# print("Model saved succesfully to :", output_dir)

Model saved succesfully to : /content/model-last


In [ ]:
nlp = spacy.load("./output/model-last/")

if not nlp:
    print("Model is not loaded...")
else:
    text_list = ["The patient underwent surgery for gallbladder removal ICD code N52, scheduled for tomorrow to address recurrent episodes of cholecystitis.",
                 "The patient underwent surgery for knee replacement  M17 and was prescribed a knee immobilizer  L1832 for post-operative support.",
                 "Following the outpatient procedure for cataract removal  66984, the patient was provided with V2799, a protective eye shield, for post-operative care."]
    for text in text_list:
      doc = nlp(text)
      for ent in doc.ents:
          print(f"Word -> {ent.text} -------- Label -> {ent.label_}")

Word -> N52 -------- Label -> ICD_CODE
Word -> M17 -------- Label -> ICD_CODE
Word -> L1832 -------- Label -> HCPCS_CODE
Word -> 66984 -------- Label -> CPT_CODE
